In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


C:\Users\Mudassir\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


In [3]:
# load dataset
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]




In [4]:
X

array([[0.02  , 0.0371, 0.0428, ..., 0.0084, 0.009 , 0.0032],
       [0.0453, 0.0523, 0.0843, ..., 0.0049, 0.0052, 0.0044],
       [0.0262, 0.0582, 0.1099, ..., 0.0164, 0.0095, 0.0078],
       ...,
       [0.0522, 0.0437, 0.018 , ..., 0.0138, 0.0077, 0.0031],
       [0.0303, 0.0353, 0.049 , ..., 0.0079, 0.0036, 0.0048],
       [0.026 , 0.0363, 0.0136, ..., 0.0036, 0.0061, 0.0115]])

In [5]:
Y

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M

In [6]:
from sklearn import preprocessing
le= preprocessing.LabelEncoder()
le.fit(Y)
en_y=le.transform(Y)

In [7]:
en_y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [8]:
# baseline model
def create_baseline():
	# create model, write code below
        model=Sequential()
        model.add(Dense(60,input_dim=60,kernel_initializer='normal',activation='relu'))
        model.add(Dense(1,activation='sigmoid'))
        
	# Compile model, write code below
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model


In [9]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, en_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 81.28% (7.81%)


In [12]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, en_y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 85.59% (4.22%)


In [25]:
# smaller model
def create_smaller():
	# create model
        model=Sequential()
        model.add(Dense(30,input_dim=60,kernel_initializer='normal',activation='relu'))
        model.add(Dense(1,activation='sigmoid'))
	# Compile model
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=250, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, en_y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 85.56% (5.23%)


In [29]:
# larger model
def create_larger():
	# create model
            model=Sequential()
            model.add(Dense(60,input_dim=60,kernel_initializer='normal',activation='relu'))
            model.add(Dense(30,activation='relu'))
            model.add(Dense(1,activation='sigmoid'))
	# Compile model
            model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
            return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, en_y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Epoch 1/100
186/186 [==============================] - 5s 28ms/step - loss: 0.6196 - acc: 0.6828
Epoch 2/100
186/186 [==============================] - 0s 919us/step - loss: 0.4433 - acc: 0.8441
Epoch 3/100
186/186 [==============================] - 0s 962us/step - loss: 0.3129 - acc: 0.9355
Epoch 4/100
186/186 [==============================] - 0s 952us/step - loss: 0.2238 - acc: 0.9516
Epoch 5/100
186/186 [==============================] - 0s 919us/step - loss: 0.1628 - acc: 0.9570
Epoch 6/100
186/186 [==============================] - 0s 946us/step - loss: 0.1189 - acc: 0.9839
Epoch 7/100
186/186 [==============================] - 0s 1ms/step - loss: 0.0852 - acc: 0.9946
Epoch 8/100
186/186 [==============================] - 0s 968us/step - loss: 0.0573 - acc: 1.0000
Epoch 9/100
186/186 [==============================] - 0s 919us/step - loss: 0.0394 - acc: 1.0000
Epoch 10/100
186/186 [==============================] - 0s 914us/step - loss: 0.0282 - acc: 1.0000
Epoch 11/100
186/186 [

187/187 [==============================] - 0s 984us/step - loss: 1.2933e-04 - acc: 1.0000
Epoch 63/100
187/187 [==============================] - 0s 877us/step - loss: 1.2431e-04 - acc: 1.0000
Epoch 64/100
187/187 [==============================] - 0s 840us/step - loss: 1.1903e-04 - acc: 1.0000
Epoch 65/100
187/187 [==============================] - 0s 888us/step - loss: 1.1354e-04 - acc: 1.0000
Epoch 66/100
187/187 [==============================] - 0s 866us/step - loss: 1.0876e-04 - acc: 1.0000
Epoch 67/100
187/187 [==============================] - 0s 898us/step - loss: 1.0392e-04 - acc: 1.0000
Epoch 68/100
187/187 [==============================] - 0s 1ms/step - loss: 1.0015e-04 - acc: 1.0000
Epoch 69/100
187/187 [==============================] - 0s 968us/step - loss: 9.5793e-05 - acc: 1.0000
Epoch 70/100
187/187 [==============================] - 0s 872us/step - loss: 9.1986e-05 - acc: 1.0000
Epoch 71/100
187/187 [==============================] - 0s 914us/step - loss: 8.8431e-05

187/187 [==============================] - 0s 1ms/step - loss: 0.0040 - acc: 1.0000
Epoch 23/100
187/187 [==============================] - 0s 840us/step - loss: 0.0036 - acc: 1.0000
Epoch 24/100
187/187 [==============================] - 0s 931us/step - loss: 0.0032 - acc: 1.0000
Epoch 25/100
187/187 [==============================] - 0s 893us/step - loss: 0.0028 - acc: 1.0000
Epoch 26/100
187/187 [==============================] - 0s 866us/step - loss: 0.0026 - acc: 1.0000
Epoch 27/100
187/187 [==============================] - 0s 856us/step - loss: 0.0023 - acc: 1.0000
Epoch 28/100
187/187 [==============================] - 0s 856us/step - loss: 0.0021 - acc: 1.0000
Epoch 29/100
187/187 [==============================] - 0s 845us/step - loss: 0.0019 - acc: 1.0000
Epoch 30/100
187/187 [==============================] - 0s 888us/step - loss: 0.0017 - acc: 1.0000
Epoch 31/100
187/187 [==============================] - 0s 856us/step - loss: 0.0016 - acc: 1.0000
Epoch 32/100
187/187 [===

187/187 [==============================] - 0s 973us/step - loss: 1.0372e-04 - acc: 1.0000
Epoch 83/100
187/187 [==============================] - 0s 872us/step - loss: 1.0111e-04 - acc: 1.0000
Epoch 84/100
187/187 [==============================] - 0s 914us/step - loss: 9.7469e-05 - acc: 1.0000
Epoch 85/100
187/187 [==============================] - 0s 877us/step - loss: 9.3595e-05 - acc: 1.0000
Epoch 86/100
187/187 [==============================] - 0s 845us/step - loss: 9.0370e-05 - acc: 1.0000
Epoch 87/100
187/187 [==============================] - 0s 834us/step - loss: 8.7721e-05 - acc: 1.0000
Epoch 88/100
187/187 [==============================] - 0s 914us/step - loss: 8.4550e-05 - acc: 1.0000
Epoch 89/100
187/187 [==============================] - 0s 872us/step - loss: 8.2125e-05 - acc: 1.0000
Epoch 90/100
187/187 [==============================] - 0s 829us/step - loss: 7.9326e-05 - acc: 1.0000
Epoch 91/100
187/187 [==============================] - 0s 872us/step - loss: 7.6980e-

187/187 [==============================] - 0s 797us/step - loss: 5.6859e-04 - acc: 1.0000
Epoch 44/100
187/187 [==============================] - 0s 759us/step - loss: 5.3287e-04 - acc: 1.0000
Epoch 45/100
187/187 [==============================] - 0s 765us/step - loss: 5.0349e-04 - acc: 1.0000
Epoch 46/100
187/187 [==============================] - 0s 770us/step - loss: 4.7354e-04 - acc: 1.0000
Epoch 47/100
187/187 [==============================] - 0s 791us/step - loss: 4.4973e-04 - acc: 1.0000
Epoch 48/100
187/187 [==============================] - 0s 781us/step - loss: 4.2371e-04 - acc: 1.0000
Epoch 49/100
187/187 [==============================] - 0s 775us/step - loss: 3.9995e-04 - acc: 1.0000
Epoch 50/100
187/187 [==============================] - 0s 781us/step - loss: 3.7835e-04 - acc: 1.0000
Epoch 51/100
187/187 [==============================] - 0s 781us/step - loss: 3.6306e-04 - acc: 1.0000
Epoch 52/100
187/187 [==============================] - 0s 754us/step - loss: 3.4319e-

188/188 [==============================] - 0s 910us/step - loss: 0.3257 - acc: 0.8777
Epoch 4/100
188/188 [==============================] - 0s 840us/step - loss: 0.2403 - acc: 0.9149
Epoch 5/100
188/188 [==============================] - 0s 1ms/step - loss: 0.1852 - acc: 0.9521
Epoch 6/100
188/188 [==============================] - 0s 904us/step - loss: 0.1350 - acc: 0.9734
Epoch 7/100
188/188 [==============================] - 0s 851us/step - loss: 0.0984 - acc: 0.9894
Epoch 8/100
188/188 [==============================] - 0s 867us/step - loss: 0.0728 - acc: 1.0000
Epoch 9/100
188/188 [==============================] - 0s 968us/step - loss: 0.0511 - acc: 0.9947
Epoch 10/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0375 - acc: 1.0000
Epoch 11/100
188/188 [==============================] - 0s 984us/step - loss: 0.0271 - acc: 1.0000
Epoch 12/100
188/188 [==============================] - 0s 995us/step - loss: 0.0192 - acc: 1.0000
Epoch 13/100
188/188 [===========

188/188 [==============================] - 0s 856us/step - loss: 1.5036e-04 - acc: 1.0000
Epoch 65/100
188/188 [==============================] - 0s 979us/step - loss: 1.4402e-04 - acc: 1.0000
Epoch 66/100
188/188 [==============================] - 0s 894us/step - loss: 1.3711e-04 - acc: 1.0000
Epoch 67/100
188/188 [==============================] - 0s 915us/step - loss: 1.3195e-04 - acc: 1.0000
Epoch 68/100
188/188 [==============================] - 0s 894us/step - loss: 1.2640e-04 - acc: 1.0000
Epoch 69/100
188/188 [==============================] - 0s 856us/step - loss: 1.2285e-04 - acc: 1.0000
Epoch 70/100
188/188 [==============================] - 0s 957us/step - loss: 1.1691e-04 - acc: 1.0000
Epoch 71/100
188/188 [==============================] - 0s 910us/step - loss: 1.1328e-04 - acc: 1.0000
Epoch 72/100
188/188 [==============================] - 0s 968us/step - loss: 1.0859e-04 - acc: 1.0000
Epoch 73/100
188/188 [==============================] - 0s 1ms/step - loss: 1.0459e-04

In [39]:
import keras
from keras import Model
from keras import layers
from keras.layers import Input
def FuncApi():
	# create model
        inputs=keras.Input(shape=(60,))
        x=layers.Dense(60,activation='relu')(inputs)
        outputs=layers.Dense(1,activation='sigmoid')(x)
	# Compile model
        model=keras.Model(inputs,outputs)
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
numpy.random.seed(seed)    
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('classifier', KerasClassifier(build_fn=FuncApi, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, en_y, cv=kfold)
print("Func_APi: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
186/186 [==============================] - 7s 38ms/step - loss: 0.5756 - acc: 0.7312
Epoch 2/100
186/186 [==============================] - 0s 1ms/step - loss: 0.4220 - acc: 0.8226
Epoch 3/100
186/186 [==============================] - 0s 995us/step - loss: 0.3476 - acc: 0.8710
Epoch 4/100
186/186 [==============================] - 0s 1ms/step - loss: 0.2939 - acc: 0.9140
Epoch 5/100
186/186 [==============================] - 0s 1ms/step - loss: 0.2512 - acc: 0.9355
Epoch 6/100
186/186 [==============================] - 0s 1ms/step - loss: 0.2208 - acc: 0.9462
Epoch 7/100
186/186 [==============================] - 0s 1ms/step - loss: 0.1917 - acc: 0.9624
Epoch 8/100
186/186 [==============================] - 0s 1ms/step - loss: 0.1695 - acc: 0.9892
Epoch 9/100
186/186 [==============================] - 0s 1ms/step - loss: 0.1508 - acc: 0.9892
Epoch 10/100
186/186 [==============================] - 0s 1ms/step - loss: 0.1311 - acc: 0.9946
Epoch 11/100
186/186 [==============

187/187 [==============================] - 0s 1ms/step - loss: 0.0027 - acc: 1.0000
Epoch 67/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0026 - acc: 1.0000
Epoch 68/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0025 - acc: 1.0000
Epoch 69/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0024 - acc: 1.0000
Epoch 70/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0023 - acc: 1.0000
Epoch 71/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0022 - acc: 1.0000
Epoch 72/100
187/187 [==============================] - 0s 1000us/step - loss: 0.0022 - acc: 1.0000
Epoch 73/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0021 - acc: 1.0000
Epoch 74/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0020 - acc: 1.0000
Epoch 75/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0019 - acc: 1.0000
Epoch 76/100
187/187 [==================

Epoch 33/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0189 - acc: 1.0000
Epoch 34/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0177 - acc: 1.0000
Epoch 35/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0164 - acc: 1.0000
Epoch 36/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0157 - acc: 1.0000
Epoch 37/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0145 - acc: 1.0000
Epoch 38/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0138 - acc: 1.0000
Epoch 39/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0130 - acc: 1.0000
Epoch 40/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0121 - acc: 1.0000
Epoch 41/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0114 - acc: 1.0000
Epoch 42/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0108 - acc: 1.0000
Epoch 43/100
187/187 [========

187/187 [==============================] - 0s 1ms/step - loss: 0.0012 - acc: 1.0000
Epoch 99/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0011 - acc: 1.0000
Epoch 100/100
21/21 [==============================] - 3s 127ms/step
Epoch 1/100
187/187 [==============================] - 8s 41ms/step - loss: 0.6949 - acc: 0.6203
Epoch 2/100
187/187 [==============================] - 0s 1ms/step - loss: 0.4901 - acc: 0.7594
Epoch 3/100
187/187 [==============================] - 0s 1ms/step - loss: 0.4008 - acc: 0.8289
Epoch 4/100
187/187 [==============================] - 0s 1ms/step - loss: 0.3424 - acc: 0.8663
Epoch 5/100
187/187 [==============================] - 0s 1ms/step - loss: 0.2982 - acc: 0.9091
Epoch 6/100
187/187 [==============================] - 0s 2ms/step - loss: 0.2624 - acc: 0.9305
Epoch 7/100
187/187 [==============================] - 0s 1ms/step - loss: 0.2318 - acc: 0.9358
Epoch 8/100
187/187 [==============================] - 0s 1ms/step - loss: 0.

187/187 [==============================] - 0s 1ms/step - loss: 0.0032 - acc: 1.0000
Epoch 65/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0030 - acc: 1.0000
Epoch 66/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0029 - acc: 1.0000
Epoch 67/100
187/187 [==============================] - 0s 1000us/step - loss: 0.0028 - acc: 1.0000
Epoch 68/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0027 - acc: 1.0000
Epoch 69/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0026 - acc: 1.0000
Epoch 70/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0025 - acc: 1.0000
Epoch 71/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0024 - acc: 1.0000
Epoch 72/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0023 - acc: 1.0000
Epoch 73/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0022 - acc: 1.0000
Epoch 74/100
187/187 [==================

188/188 [==============================] - 0s 979us/step - loss: 0.0201 - acc: 1.0000
Epoch 31/100
188/188 [==============================] - 0s 979us/step - loss: 0.0185 - acc: 1.0000
Epoch 32/100
188/188 [==============================] - 0s 1000us/step - loss: 0.0173 - acc: 1.0000
Epoch 33/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0160 - acc: 1.0000
Epoch 34/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0150 - acc: 1.0000
Epoch 35/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0141 - acc: 1.0000
Epoch 36/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0131 - acc: 1.0000A: 0s - loss: 0.0111 - acc: 1.00
Epoch 37/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0125 - acc: 1.0000
Epoch 38/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0117 - acc: 1.0000
Epoch 39/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0109 - acc: 1.0000
Epoc

188/188 [==============================] - 0s 952us/step - loss: 0.0012 - acc: 1.0000
Epoch 96/100
188/188 [==============================] - 0s 947us/step - loss: 0.0011 - acc: 1.0000
Epoch 97/100
188/188 [==============================] - 0s 979us/step - loss: 0.0011 - acc: 1.0000
Epoch 98/100
188/188 [==============================] - 0s 1ms/step - loss: 0.0011 - acc: 1.0000
Epoch 99/100
188/188 [==============================] - 0s 995us/step - loss: 0.0010 - acc: 1.0000
Epoch 100/100
20/20 [==============================] - 3s 157ms/step
Func_APi: 84.59% (6.01%)


In [49]:
import tensorflow as tf
def create_baseline_3():
    class MyModel(tf. keras.Model):
        def __init__(self):
            super(MyModel,self).__init__()
            self.dense1 = Dense(60, activation="relu")
            self.dense2 = Dense(10, activation='relu')
            self.dense3 = Dense(1, activation='sigmoid')


        def call(self,inputs):
            x = self.dense1(inputs)
            x = self.dense2(x)
            return self.dense3(x)

    model=MyModel()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model




numpy.random.seed(seed)
estimators=[]
estimators.append(('standardize',StandardScaler()))
estimators.append(('classifier',KerasClassifier(build_fn=create_baseline_3,epochs=100,batch_size=5,verbose=0)))
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
pipeline=Pipeline(estimators)
results=cross_val_score(pipeline,X,en_y,cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 54.27% (10.44%)


In [73]:
import numpy as np
k=10
num_val_samples = len(X) // k
num_epochs = 100
val_loss=[]
val_acc=[]
np.random.seed(seed)
for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = en_y[i* num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate([X[:i * num_val_samples],
                                         X[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = np.concatenate([en_y[:i*num_val_samples],
                                            en_y[(i + 1) * num_val_samples:]],axis=0)
    
    model=create_baseline()
    history=model.fit(partial_train_data,partial_train_targets,shuffle=True,epochs=100,
                      batch_size=5,verbose=0)
    sys=model.evaluate(val_data,val_targets,verbose=0)
    val_loss.append(sys[0])
    val_acc.append(sys[1])

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [74]:
len(val_acc)

10

In [75]:
np.mean(val_acc)

0.5850000072270631

In [76]:
np.mean(val_loss)

0.7176927894353866